In [1]:
import pandas as pd
import numpy as np
import json
import openai
import re 

openai.api_key = #

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

In [3]:

import itertools

def search_primary_key_from_df(df, candidates_cols=None, na_threshold=0, max_dim=None):
    # 默认所有列为候选
    if candidates_cols is None:
        candidates_cols = df.columns
    if max_dim is None:
        max_dim = len(candidates_cols) - 1
    
    # 排除不合法的列，即数据类型不是数值或字符串的列，或na值比例超过na_threshold的列
    valid_cols = []
    for col in candidates_cols:
        if df[col].dtype == float:
            continue
        na_percentage = df[col].isna().sum() / df.shape[0]
        if na_percentage <= na_threshold:
            valid_cols.append(col)
    
    # primary key搜索
    results = []
    for i in range(1, max_dim+1):
        for cols in itertools.combinations(valid_cols, i):
            unique_values = df[list(cols)].drop_duplicates().shape[0]
            if unique_values == df.shape[0]:
                results.append((list(cols)))
        if results:
            break
    
    return results

def dataframe_to_markdown(df):
    # 获取列名和数据
    columns = df.columns
    data = df.values.tolist()

    # 构建Markdown表格
    markdown_table = "| " + " | ".join(columns) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * len(columns)) + " |\n"

    for row in data:
        markdown_table += "| " + " | ".join(map(str, row)) + " |\n"

    return markdown_table


def map_data_type(dtype):
    if np.issubdtype(dtype, np.number):
        return 'Numeric'
    elif np.issubdtype(dtype, np.datetime64):
        return 'DateTime'
    elif np.issubdtype(dtype, np.bool_):
        return 'Boolean'
    elif np.issubdtype(dtype, np.object_):
        return 'String'
    else:
        return 'Unknown'

def compute_dataframe_stats(df):
    stats = []
    
    for column in df.columns:
        column_stats = {}
        column_stats['field'] = column
        column_stats['type'] = map_data_type(df[column].dtype)
        if column_stats['type'] in ['Numeric', 'DateTime']:
            column_stats['max'] = df[column].max()
            column_stats['min'] = df[column].min()
        column_stats['uniques'] = df[column].nunique()
        column_stats['nan_percentage'] = df[column].isnull().mean() * 100
        stats.append(column_stats)
    
    return pd.DataFrame(stats)

In [4]:
data_info = f"""
dataset name: 行业市场变化_市场大盘分析_行业市场大盘_魔镜

dataset description:
primary keys:[月份, 类目编码]
3540 rows
| field | type | uniques | nan_percentage | max | min |
| --- | --- | --- | --- | --- | --- |
| 月份 | DateTime | 59 | 0.0 | nan | nan |
| 平台编码 | String | 2 | 0.0 | nan | nan |
| 一级类目 | String | 4 | 0.0 | nan | nan |
| 二级类目 | String | 8 | 6.666666666666667 | nan | nan |
| 三级类目 | String | 16 | 20.0 | nan | nan |
| 四级类目 | String | 32 | 46.666666666666664 | nan | nan |
| 类目名称 | String | 60 | 0.0 | nan | nan |
| 类目编码 | Numeric | 60 | 0.0 | 39264.0 | 10209.0 |
| 销售额 | Numeric | 3540 | 0.0 | 99989.30547 | 1.8873 |
| 销量 | Numeric | 974 | 0.0 | 999.0 | 0.0 |
| 店铺数 | Numeric | 181 | 0.0 | 200.0 | 20.0 |
| 商品数 | Numeric | 451 | 0.0 | 500.0 | 50.0 |
| 收藏次数 | Numeric | 1 | 0.0 | 0.0 | 0.0 |
"""


def chat(prompt):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=[
        {"role": "system", "content": f"""你是一个数据分析机器人，用户会向你咨询关于数据的问题，请按照以下步骤进行处理。
        1. 需要判断用户的问题是否为这两类【常规数据咨询，BA看板】，对于这两个问题之外的，请礼貌的回答无法满足要求。
            想知道具体单一数值或简单问题的，为数据咨询；若用户想知道多个数据的占比或趋势或涉及多个数据维度之间的关系，属于BA看板。
        2A. 如果为数据咨询，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以打印出回答此问题的信息。
        2B. 如果为BA看板，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以绘制对应需求。
        3. 以JSON格式返回信息。Ensure the response can be parsed by Python json.loads。Json外不能有任何字符
        "message":<所有需要沟通的内容>, "task":<问题分类>, "python_code":<python code>
        
        要求：写python代码时需假设df已经读取在内存中，你不需要写读取数据的代码。
        
        你掌握的数据情况如下：
        {data_info}    
        """},
        {"role": "user", "content": prompt},
    ])
    
    
    return response, response['choices'][0]['message']['content']

In [5]:
df = pd.read_csv('行业市场变化_市场大盘分析_行业市场大盘_魔镜.csv')
df = df.drop(['平台名称', 'ETL更新时间'], axis=1)
df['月份'] = pd.to_datetime(df['月份'])

In [6]:
res, content = chat('目前销量最好的品类是哪个？')
print(content)

{
    "message": "根据数据集中销量字段，可以按照类目编码进行分组求和，然后按照销量降序排序，得到销量最好的品类是类目编码为 39264 的品类。",
    "task": "常规数据咨询",
    "python_code": "df.groupby('类目编码')['销量'].sum().sort_values(ascending=False).index[0]"
}


In [7]:
python_code = json.loads(content)['python_code']
print(python_code)
exec(python_code)

df.groupby('类目编码')['销量'].sum().sort_values(ascending=False).index[0]
